In [31]:
import numpy as np
import pandas as pd
from fractions import Fraction
import os 
groundState = -1.5746

In [13]:
def build_kitaev_matrix(N, wp, t, ep, g, Jx=1.0, Jy=1.0, Jz=1.0):
    # Total number of sites (2 per unit cell)
    num_sites = 2 * N * N
#     plack = np.zeros(N*N)
    
#     for idx,i in enumerate (plack):
#         if idx%4 == 0:
#             plack[idx] = 1
#         else:
#             plack[idx] = 0
#     assert np.sum(plack)%2 == 0
    # Initialize the M matrixes
    M_hex = 1j*np.zeros((num_sites, num_sites))
    M_tup = 1j*np.zeros((num_sites//2, num_sites//2))
    M_tdown = 1j*np.zeros((num_sites//2, num_sites//2))
    
    zlist, xlist = generateLists(wp,N,N)
    #print(xlist)
    #print(zlist)
    
    # Loop over unit cells for hex matrix
    for m in range(N):
        for n in range(N):
            # Site indices for A and B in unit cell (m, n) where m is the row index and n is the collum index
            idx_A = (m * N + n) * 2  # Sublattice A
            idx_B = idx_A + 1        # Sublattice B

            # x-bond: A(m,n) to B(m,n) IF INSWET
            if idx_A  in xlist: 
                M_hex[idx_A, idx_B] = -1*-2*Jx*1j
                M_hex[idx_B, idx_A] = -1*2*Jx*1j  
            else: 
                M_hex[idx_A, idx_B] = -2*Jx*1j
                M_hex[idx_B, idx_A] = 2*Jx*1j

            # y-bond: B(m,n) to A(m, n+1)
            n_next = (n + 1) % N
            idx_A_next = (m * N + n_next) * 2
            M_hex[idx_B, idx_A_next] = 2*Jy*1j
            M_hex[idx_A_next, idx_B] = -2*Jy*1j

            # z-bond: B(m,n) to A(m+1, n) IF INSWERT
            m_next = (m - 1) % N
            n_next = (n+1) %N 
            idx_A_next = (m_next * N + n_next) * 2
            if idx_A_next in zlist: 
                M_hex[idx_B, idx_A_next] = -1*2*Jz*1j
                M_hex[idx_A_next, idx_B] = -1*-2*Jz*1j
            else: 
                M_hex[idx_B, idx_A_next] = 2*Jz*1j
                M_hex[idx_A_next, idx_B] = -2*Jz*1j

                
    #loop over unit cells for trig matrix            
    for m in range(N):
        for n in range(N):
            # Site indices for A and B in unit cell (m, n) where m is the row index and n is the collum index
    #             idx_A = (m * N + n) * 2  # Sublattice A
    #             idx_B = idx_A + 1        # Sublattice B
            idx = m*N + n
            idx_x = (m)*N + (n+1)%N
            idx_y = ((m+1)%N)*N + (n)
            idx_z = ((m-1)%N)*N + (n+1)%N
            #x-bond 
            M_tup[idx,idx_x] += -t
            M_tup[idx_x,idx] += -t
            
            M_tdown[idx,idx_x] += -t
            M_tdown[idx_x,idx] += -t
            
            #ybond
            M_tup[idx,idx_y] += -t
            M_tup[idx_y,idx] += -t
            
            M_tdown[idx,idx_y] += -t
            M_tdown[idx_y,idx] += -t
            
            #z-bond
            M_tup[idx,idx_z] += -t
            M_tup[idx_z,idx] += -t
            
            M_tdown[idx,idx_z] += -t
            M_tdown[idx_z,idx] += -t
            
            #self 
#             print(f"  the length of our placket is {len(wp)}, the legnth of up and down is{len(M_tup)}, {len(M_tdown)}, and the idx is {idx}")
            if len(wp)==0:
                print(wp)
            if wp[idx] == 0: 
                M_tup[idx,idx] += g+ep
                M_tdown[idx,idx] += -g+ep
            if wp[idx] == 1:
                M_tup[idx,idx] += -g+ep
                M_tdown[idx,idx] += g+ep

    
    return M_hex,M_tup,M_tdown

In [14]:
def generateLists (plackList,nx,ny):
    PlackPairList = []
    idx1 = -1
    idx2= -1
    zlist = []
    xlist = []
    #print(len(plackList))
    for idx,plack in enumerate(plackList):
        #print(idx)
        if plack == 1 and idx1 == -1 and idx2 == -1:
            idx1 = idx
        elif plack == 1 and idx1 != -1:
            PlackPairList.append((idx1,idx))
            idx1 = -1 
    #print(plackList,PlackPairList)
    for p1,p2 in PlackPairList:
        r1= p1//ny
        r2= p2//ny
        c1= p1%ny
        c2= p2%ny
        #print(r1,c1,r2,c2)
        site1 = (r1*2*nx)+2*c1 
        site2 = (r2*2*nx)+2*c2
        for col in range(0,c2-c1):
            zlist.append(site1+2*(1+col))
        if r2-r1 > 0:
            for row in range(0,r2-r1):
                xlist.append(site2-row*2*nx)
        elif r2-r1 < 0:
            for row in range(0,r1-r2):
                xlist.append(site2+((row+1)*2*nx))

    return zlist,xlist

In [24]:
def diagonalize_kitaev(N, wp, Jx=1.0, Jy=1.0, Jz=1.0,t= 1,ep = 1,g = 1,K = 1.0, filling = 1/2):
    M,Mup,Mdown = build_kitaev_matrix(N, wp, t, ep, g, Jx=K, Jy=K, Jz=K)
    Mhermitian = np.conj(M.T)
    assert (np.all(np.isclose(M,Mhermitian)))
    Mhermitian = np.conj(Mup.T)
    assert (np.all(np.isclose(Mup,Mhermitian)))
    Mhermitian = np.conj(Mdown.T)
    assert (np.all(np.isclose(Mdown,Mhermitian)))
    
    
    # Diagonalize M (antisymmetric, so eigenvalues are pure imaginary or zero)
    eigenvalues = np.linalg.eigvalsh(M)
    eigenvaluesUp = np.linalg.eigvalsh(Mup)
    eigenvaluesDown = np.linalg.eigvalsh(Mdown)
    
    # Sort eigenvalues by their real part
#     eigenvalues = np.sort(eigenvalues)[::-1]
#     eigenvalues = np.sort(eigenvalues)[::-1]
#     eigenvalues = np.sort(eigenvalues)[::-1]
    
    # Compute ground state energy: sum of positive eigenvalues / 2
    positive_eigenvalues = eigenvalues[eigenvalues > 0]
    upEigen = eigenvaluesUp[eigenvaluesUp < 0]
    downEigen = eigenvaluesDown[eigenvaluesDown < 0]
    upDownEigen = np.append(eigenvaluesDown , eigenvaluesUp)
    #np.partition(upDownEigen, (N*N)*filling )[(N*N)*filling]
    sortedUpDown = np.sort(upDownEigen)
    
#     ground_state_energyUp = np.sum(upEigen)
#     ground_state_energyDown = np.sum(downEigen)
    ground_state_upDownEnergy = np.sum(sortedUpDown[:int(N*N*filling)])
    ground_state_energy = -0.5 * np.sum(positive_eigenvalues)
    
    #return [eigenvalues, ground_state_energy, ground_state_energyDown,ground_state_energyUp, ground_state_energy+ground_state_energyDown+ground_state_energyUp]
    return [eigenvalues, ground_state_upDownEnergy, ground_state_energy, ground_state_energy+ground_state_upDownEnergy]

In [33]:
#basic energies use t = 0 for phase diagram t = 1
#phase diagram
t = 1
N=12
points = 10
# wpList = wpGenerator(MaxPatternSize)
placketLists = kitaevList(N,N)
cached_results = f"NVal_{N}_TVal_{t}"
# print(f"{len (placketLists)}")
SpecificWP = []
for filling_str in ["1/2","1/4","1/3"]:
    finalList = [] #make a dict where you have the key being {placket,g,k:} and then the values are something tbd
    filling_safe = filling_str.replace("/", "-")
    cached_result = cached_results + f"_Filling_{filling_safe}.csv"
    # Source - https://stackoverflow.com/a
    # Posted by newacct, modified by community. See post 'Timeline' for change history
    # Retrieved 2025-11-19, License - CC BY-SA 4.0
    filling = float(sum(Fraction(s) for s in filling_str.split()))
    #try opening the csv if it exsits
    if os.path.exists(cached_result):
        df_existing = pd.read_csv(cached_result)
        print(f"Loaded existing CSV with {len(df_existing)} rows.")
    #make a new csv if one doesnt exsit
    else:
        df_existing = pd.DataFrame(columns=["g", "k", "winning_config"])
        print("No existing CSV found, starting fresh.")
    #then  make sure that the df has right data types
    df_existing["g"] = pd.to_numeric(df_existing["g"], errors="coerce")
    df_existing["k"] = pd.to_numeric(df_existing["k"], errors="coerce")
    
    #start looping through the points
    for g in np.linspace(0.3,0.8,points): #orginally np.arange(0.3,0.8,0.005)
        for K in [0.5]:#np.arange(0.45,0.5,0.001):
            #check if the g,k pair alredy exsits
            mask = (np.isclose(df_existing["g"], g)) & (np.isclose(df_existing["k"], K))
            
            if mask.any():
                print(f"g={g:.4f}, k={K:.4f} already exists → skipping computation.")
                continue #skips to the next loop itteration
            
            print(f"Computing new point g={g:.4f}, k={K:.4f} ...")
            allWP = []
            for index,Wp in enumerate(placketLists):
                if len(Wp) == 0:
                    print (f"The weird index is {index}")
                allWP.append(diagonalize_kitaev(N, Wp, g = g , K = K, filling = filling))
                print(f"computing for g:{g} and k:{K} and wp:{index}")
                
            print(f"done computeing all wp for g,k pair {g,K} now finding the minuimum wp!" )
            minEnergy = min([gse[3] for gse in allWP ])
            minIndex = [List[3] for List in allWP].index(minEnergy)
            minPlackConfig = placketLists[minIndex]
            finalList.append ([g,K,minIndex])
            print(f"Appended min index {minIndex} for g,k value {g,K}!")
#     df = pd.DataFrame(finalList, columns=["g","k", "winning_config"])
#     df.to_csv(f"phase_results_ZoomgkpointTEST_{filling}.csv", index=False)
#     print("Saved results to phase_results.csv ✅")
    if finalList:
        df_new = pd.DataFrame(finalList, columns=["g", "k", "winning_config"])
        df_updated = pd.concat([df_existing, df_new], ignore_index=True)
        df_updated.to_csv(cached_result, index=False)
        print(f"Saved updated results to {cached_result} ✔️")
    else:
        print("No new (g,k) points needed computing — CSV unchanged.")


No existing CSV found, starting fresh.
Computing new point g=0.3000, k=0.5000 ...
computing for g:0.3 and k:0.5 and wp:0
computing for g:0.3 and k:0.5 and wp:1
computing for g:0.3 and k:0.5 and wp:2
computing for g:0.3 and k:0.5 and wp:3
computing for g:0.3 and k:0.5 and wp:4
computing for g:0.3 and k:0.5 and wp:5
computing for g:0.3 and k:0.5 and wp:6
computing for g:0.3 and k:0.5 and wp:7
computing for g:0.3 and k:0.5 and wp:8
computing for g:0.3 and k:0.5 and wp:9
computing for g:0.3 and k:0.5 and wp:10
computing for g:0.3 and k:0.5 and wp:11
computing for g:0.3 and k:0.5 and wp:12
computing for g:0.3 and k:0.5 and wp:13
computing for g:0.3 and k:0.5 and wp:14
done computeing all wp for g,k pair (0.3, 0.5) now finding the minuimum wp!
Appended min index 14 for g,k value (0.3, 0.5)!
Computing new point g=0.3556, k=0.5000 ...
computing for g:0.3555555555555555 and k:0.5 and wp:0
computing for g:0.3555555555555555 and k:0.5 and wp:1
computing for g:0.3555555555555555 and k:0.5 and wp:2

computing for g:0.7444444444444445 and k:0.5 and wp:9
computing for g:0.7444444444444445 and k:0.5 and wp:10
computing for g:0.7444444444444445 and k:0.5 and wp:11
computing for g:0.7444444444444445 and k:0.5 and wp:12
computing for g:0.7444444444444445 and k:0.5 and wp:13
computing for g:0.7444444444444445 and k:0.5 and wp:14
done computeing all wp for g,k pair (0.7444444444444445, 0.5) now finding the minuimum wp!
Appended min index 1 for g,k value (0.7444444444444445, 0.5)!
Computing new point g=0.8000, k=0.5000 ...
computing for g:0.8 and k:0.5 and wp:0
computing for g:0.8 and k:0.5 and wp:1
computing for g:0.8 and k:0.5 and wp:2
computing for g:0.8 and k:0.5 and wp:3
computing for g:0.8 and k:0.5 and wp:4
computing for g:0.8 and k:0.5 and wp:5
computing for g:0.8 and k:0.5 and wp:6
computing for g:0.8 and k:0.5 and wp:7
computing for g:0.8 and k:0.5 and wp:8
computing for g:0.8 and k:0.5 and wp:9
computing for g:0.8 and k:0.5 and wp:10
computing for g:0.8 and k:0.5 and wp:11
compu

computing for g:0.6888888888888889 and k:0.5 and wp:4
computing for g:0.6888888888888889 and k:0.5 and wp:5
computing for g:0.6888888888888889 and k:0.5 and wp:6
computing for g:0.6888888888888889 and k:0.5 and wp:7
computing for g:0.6888888888888889 and k:0.5 and wp:8
computing for g:0.6888888888888889 and k:0.5 and wp:9
computing for g:0.6888888888888889 and k:0.5 and wp:10
computing for g:0.6888888888888889 and k:0.5 and wp:11
computing for g:0.6888888888888889 and k:0.5 and wp:12
computing for g:0.6888888888888889 and k:0.5 and wp:13
computing for g:0.6888888888888889 and k:0.5 and wp:14
done computeing all wp for g,k pair (0.6888888888888889, 0.5) now finding the minuimum wp!
Appended min index 14 for g,k value (0.6888888888888889, 0.5)!
Computing new point g=0.7444, k=0.5000 ...
computing for g:0.7444444444444445 and k:0.5 and wp:0
computing for g:0.7444444444444445 and k:0.5 and wp:1
computing for g:0.7444444444444445 and k:0.5 and wp:2
computing for g:0.7444444444444445 and k:0

computing for g:0.6333333333333333 and k:0.5 and wp:3
computing for g:0.6333333333333333 and k:0.5 and wp:4
computing for g:0.6333333333333333 and k:0.5 and wp:5
computing for g:0.6333333333333333 and k:0.5 and wp:6
computing for g:0.6333333333333333 and k:0.5 and wp:7
computing for g:0.6333333333333333 and k:0.5 and wp:8
computing for g:0.6333333333333333 and k:0.5 and wp:9
computing for g:0.6333333333333333 and k:0.5 and wp:10
computing for g:0.6333333333333333 and k:0.5 and wp:11
computing for g:0.6333333333333333 and k:0.5 and wp:12
computing for g:0.6333333333333333 and k:0.5 and wp:13
computing for g:0.6333333333333333 and k:0.5 and wp:14
done computeing all wp for g,k pair (0.6333333333333333, 0.5) now finding the minuimum wp!
Appended min index 14 for g,k value (0.6333333333333333, 0.5)!
Computing new point g=0.6889, k=0.5000 ...
computing for g:0.6888888888888889 and k:0.5 and wp:0
computing for g:0.6888888888888889 and k:0.5 and wp:1
computing for g:0.6888888888888889 and k:0

In [23]:
def find_min_plackets(N, placketLists, filling=1/2, g_range=None, K_range=None):
    """ For each (g, K) in the given ranges, find the placket configuration index that minimizes the energy. Returns: g_vals: array of g values K_vals: array of K values winning_config: 2D array [len(g_vals), len(K_vals)] of indices """ 
    if g_range is None: 
        g_range = np.arange(0, 1.01, 0.1) # inclusive
    if K_range is None: 
        K_range = np.arange(0, 1.01, 0.1) 
    winning_config = np.zeros((len(g_range), len(K_range)), dtype=int) 
    for i, g in tqdm(enumerate(g_range)): 
        for j, K in enumerate(K_range): 
            allWP = [] 
            for Wp in placketLists: 
                allWP.append( diagonalize_kitaev(N, Wp, g=g, K=K, filling=filling) ) # Find min energy and index 
                energies = [gse[3] for gse in allWP] 
                min_idx = np.argmin(energies) 
                winning_config[i, j] = min_idx
                finalList.append ([g,K,minIndex])
    return finalList

In [ ]:
for filling in [1/2,1/4,1/3]:
    for step_size in np.arange(0,1,0.1):
    df = pd.DataFrame(finalList, columns=["g","k", "winning_config"])
    df.to_csv(f"phase_results_100point_{filling}.csv", index=False)
    print("Saved results to phase_results.csv ✅")


In [24]:
#redoing kitaev calculation
t = 0
g = 0
K = 1
ep = 0
N = 24
orginalTable = {}
placketLists = kitaevList(N,N)

for idx, Wp in enumerate (placketLists):
    orginalTable[idx] = (diagonalize_kitaev(N,Wp,g=g,K=K,t=t)[2]/(N*N))-groundState
print(orginalTable)
#4,5,9,11 is same as 10

2 0
2 3
2 6
2 9
2 12
2 15
2 18
2 21
5 0
5 3
5 6
5 9
5 12
5 15
5 18
5 21
8 0
8 3
8 6
8 9
8 12
8 15
8 18
8 21
11 0
11 3
11 6
11 9
11 12
11 15
11 18
11 21
14 0
14 3
14 6
14 9
14 12
14 15
14 18
14 21
17 0
17 3
17 6
17 9
17 12
17 15
17 18
17 21
20 0
20 3
20 6
20 9
20 12
20 15
20 18
20 21
23 0
23 3
23 6
23 9
23 12
23 15
23 18
23 21
{0: 0.06721430415255769, 1: 0.05213307278089818, 2: 0.04162313303383014, 3: 0.053761144014813356, 4: 0.02588289583614789, 5: 0.06010213339590531, 6: 0.034146559306408975, 7: 0.04245029847972126, 8: 0.058957137915345736, 9: 0.04163822091428182, 10: 0.07453098224099408, 11: 0.025305932291394884, 12: 0.04610307036740058, 13: 0.07217294825271381}


In [7]:
wpGenerator()

NameError: name 'wpGenerator' is not defined

In [22]:
#the convention that we use is left to right then up to down -> then v
def kitaevList(nx,ny):
    All_plack_kitev = [[] for i in range (15)]
    #we want to go through all the cols for each row so we pick a row ny, and then go thorugh all the cols nx
    for row in range(ny): 
        for col in range(nx):
            #rows and col's start at 0
            #1st phase
            All_plack_kitev[0].append(1) #all cells filled
            All_plack_kitev[14].append(0) #all cells empty
            
            #2nd phase
            if row%2 == 0:
                All_plack_kitev[1].append(1)
            if row%2 == 1:
                All_plack_kitev[1].append(0)
            
            #3rd phase
            if row%3 == 2: 
                All_plack_kitev[2].append(1)
            if row%3 == 1 or row%3 == 0:
                All_plack_kitev[2].append(0)
            
            #4th phase
            if row%3 == 1 or row%3 == 2:
                All_plack_kitev[3].append(1)
            if row%3 == 0:
                All_plack_kitev[3].append(0)
            
            #5th phase
            if row%3 == 0 and col%3 == 1:
                All_plack_kitev[4].append(1)
            elif row%3 == 1 and col%3 == 2: 
                All_plack_kitev[4].append(1)
            elif row%3 == 2 and col%3 == 0:
                All_plack_kitev[4].append(1)
#                 print(row,col)
            else:
                All_plack_kitev[4].append(0)
            
            #6th phase
            if row%3 == 0 and col %3 == 1:
                All_plack_kitev[5].append(0)
            elif row%3 == 1 and col%3 == 2: 
                All_plack_kitev[5].append(0)
            elif row%3 == 2 and col%3 == 0:
                All_plack_kitev[5].append(0)
            else:
                All_plack_kitev[5].append(1)
           
            #7th phase
            if row%4 == 0:
                All_plack_kitev[6].append(1)
            else:
                All_plack_kitev[6].append(0)
            
            #8th phase
            if row% 4 == 0 or row%4 == 3:
                All_plack_kitev[7].append(1)
            else: 
                All_plack_kitev[7].append(0)
            
            #9th phase
            if row%4 == 1: 
                All_plack_kitev[8].append(0)
            else:
                All_plack_kitev[8].append(1)
            
            #10th phase 
            if row%2 == 0: 
                All_plack_kitev[9].append(0)
            else:
                if col %2 == 0:
                    All_plack_kitev[9].append(0)
                else:
                    All_plack_kitev[9].append(1)
                    
            
            #11th phase
            if row%2 == 0:
                All_plack_kitev[10].append(1)
            else: 
                if col%2 == 0:
                    All_plack_kitev[10].append(1)
                else:
                    All_plack_kitev[10].append(0)
            
            #12th phase 
            if row%4 == 1 or row%4 == 3:
                All_plack_kitev[11].append(0)
            elif row%4 == 0:
                if col%2 ==1:
                    All_plack_kitev[11].append(0)
                else:
                    All_plack_kitev[11].append(1)
            else:
                if col%2 == 0: 
                    All_plack_kitev[11].append(0)
                else:
                    All_plack_kitev[11].append(1)
            
            #13th phase
            if row%4 == 0 or row%4 == 3:
                if col % 2 == 0:
                    All_plack_kitev[12].append(0)
                else:
                    All_plack_kitev[12].append(1)
            elif row%4 == 1 or row%4 == 2:
                if col %2 == 0:
                    All_plack_kitev[12].append(1)
                else:
                    All_plack_kitev[12].append(0)
            
            #14th phase
            if row%4 == 1 or row%4 == 3:
                All_plack_kitev[13].append(1)
            elif row%4 == 0:
                if col%2 ==1:
                    All_plack_kitev[13].append(1)
                else:
                    All_plack_kitev[13].append(0)
            else:
                if col%2 == 0: 
                    All_plack_kitev[13].append(1)
                else:
                    All_plack_kitev[13].append(0)
    return All_plack_kitev

In [19]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import RegularPolygon
from matplotlib.colors import ListedColormap

def plot_kitaev_patterns_hex(nx, ny):
    patterns = kitaevList(nx, ny)

    # Define two colors: 0 = yellow, 1 = blue
    cmap = ListedColormap(["yellow", "blue"])

    fig, axes = plt.subplots(2, 7, figsize=(22, 8))  # 14 phases
    axes = axes.ravel()

    # Hex geometry
    radius = 0.5
    dx = 3/2 * radius       # horizontal spacing
    dy = np.sqrt(3) * radius  # vertical spacing

    for phase, pattern in enumerate(patterns):
        arr = np.array(pattern).reshape(ny, nx)

        ax = axes[phase]
        ax.set_aspect("equal")
        ax.axis("off")
        ax.set_title(f"Phase {phase+1}")

        for row in range(ny):
            for col in range(nx):
                # offset every other row
                x = col * dx
                if row % 2 == 1:
                    x += dx / 2
                y = row * dy 

                color = cmap(arr[row, col])
                hexagon = RegularPolygon(
                    (x, y), numVertices=6, radius=radius,
                    orientation=np.radians(30),
                    facecolor=color, edgecolor="k"
                )
                ax.add_patch(hexagon)

        ax.set_xlim(-1, nx*dx + 1)
        ax.set_ylim(-1, ny*dy*0.5 + 1)

    plt.tight_layout()
    plt.show()


In [10]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import RegularPolygon
from matplotlib.colors import ListedColormap

def plot_kitaev_patterns_hex_1(nx, ny):
    patterns = kitaevList(nx, ny)

    # Custom 2-color map: 0 = yellow, 1 = blue
    cmap = ListedColormap(["yellow", "blue"])

    fig, axes = plt.subplots(2, 7, figsize=(22, 8))  # 14 phases
    axes = axes.ravel()

    # Hexagon geometry
    dx = 1.0          # horizontal spacing
    dy = np.sqrt(3)/2 # vertical spacing (for hex tiling)

    for phase, pattern in enumerate(patterns):
        arr = np.array(pattern).reshape(ny, nx)

        ax = axes[phase]
        ax.set_aspect("equal")
        ax.axis("off")
        ax.set_title(f"Phase {phase+1}")

        # Plot hexagons
        for row in range(ny):
            for col in range(nx):
                x = col + 0.5 * (row % 2)  # stagger every other row
                y = row * dy
                color = cmap(arr[row, col])
                hexagon = RegularPolygon(
                    (x, y), numVertices=6, radius=0.5, orientation=np.radians(30),
                    facecolor=color, edgecolor="k"
                )
                ax.add_patch(hexagon)

        ax.set_xlim(-1, nx+1)
        ax.set_ylim(-1, ny*dy+1)

    plt.tight_layout()
    plt.show()


In [ ]:
plot_kitaev_patterns_hex_1(12,12)